In [2]:
import os
os.chdir('..')

In [ ]:
import torch as th

In [ ]:
len(th.optim.Optimizer.__subclasses__())

In [ ]:
def all_subclasses(cls):
    return set(cls.__subclasses__()).union(
        [s for c in cls.__subclasses__() for s in all_subclasses(c)])

In [ ]:
len(all_subclasses(th.optim.Optimizer))

In [ ]:
from FLF.TorchOptRepo import TorchOptRepo

In [ ]:
{opt: TorchOptRepo.supported_parameters(opt) for opt in TorchOptRepo.get_opt_names()}

In [ ]:
from FLF.TorchFederatedLearnerMNIST import Net

In [ ]:
model = Net()

In [ ]:
opt = TorchOptRepo.name2cls('Adam')(model.parameters())

In [ ]:
dir(opt)

In [ ]:
from comet_ml import Experiment
import logging
from FLF.TorchFederatedLearnerMNIST import (
    TorchFederatedLearnerMNIST,
    TorchFederatedLearnerMNISTConfig,
)
from FLF.TorchOptRepo import TorchOptRepo

logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S",
)


def train_fakes(opt):
    C = 1
    NC = 2
    E = 1
    B = 50
    is_iid = False
    lr = 0.000001
    name = f"fake"

    logging.info(name)
    experiment = Experiment(workspace="federated-learning", project_name="2C_opt")
    experiment.set_name(name)
    experiment.log_parameter("opt_srategy", "reinit")
    # TODO a paraméterek helytelen nevére nem adott hibát
    config = TorchFederatedLearnerMNISTConfig(
        LEARNING_RATE=lr,
        OPT=opt,
        IS_IID_DATA=is_iid,
        BATCH_SIZE=B,
        CLIENT_FRACTION=C,
        N_CLIENTS=NC,
        N_EPOCH_PER_CLIENT=E,
        MAX_ROUNDS=1,
    )
    learner = TorchFederatedLearnerMNIST(experiment, config)
    learner.train()
    return learner

learners = {opt: train_fakes(opt) for opt in TorchOptRepo.get_opt_names()}

In [ ]:
learner.clients[0].opt.state_dict()['state']

In [102]:
new_opt = TorchOptRepo.name2cls("Adam")(learners["Adam"].model.parameters())

In [124]:
new_opt.state_dict()

{'state': {},
 'param_groups': [{'lr': 0.001,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'params': [140520058830272,
    140520058830352,
    140520087878784,
    140520058829152,
    140520058826992,
    140520058830672,
    140520058827872,
    140520087165840]}]}

In [114]:
new_state_dict = new_opt.state_dict()#['param_groups'][0]['params']

In [122]:
new_state_dict['state'].update(zip(new_state_dict['param_groups'][0]['params'], learners["Adam"].clients[0].opt.state_dict()['state'].values()))

In [125]:
new_opt.load_state_dict(new_state_dict)

In [126]:
new_opt.state_dict()

{'state': {140520058830272: {'step': 600,
   'exp_avg': tensor([[[[-1.2564e-02, -1.2012e-02, -2.7444e-03,  3.3981e-03,  4.2544e-03],
             [-1.7731e-02, -1.9015e-02, -1.1222e-02, -2.7475e-03, -2.6343e-03],
             [-1.6720e-02, -2.0777e-02, -2.0666e-02, -1.6326e-02, -1.2824e-02],
             [-1.3607e-02, -1.9284e-02, -2.2210e-02, -2.0273e-02, -1.5832e-02],
             [-1.3135e-02, -1.2960e-02, -1.6214e-02, -1.3896e-02, -9.5490e-03]]],
   
   
           [[[-1.3942e-03, -2.8013e-03, -1.1715e-03, -2.9546e-04, -4.3434e-03],
             [-3.1431e-03, -2.4931e-03, -3.6715e-04, -2.5569e-04, -5.0479e-03],
             [-3.9029e-03, -3.0813e-03, -1.6875e-03, -2.3630e-03, -6.7164e-03],
             [-4.2328e-03, -3.9391e-03, -3.4622e-03, -6.6556e-03, -9.8460e-03],
             [-4.2582e-03, -4.9635e-03, -4.9212e-03, -1.0972e-02, -1.1856e-02]]],
   
   
           [[[ 2.0595e-03,  3.1185e-03,  3.7564e-03,  2.6698e-03,  2.7187e-03],
             [ 2.0446e-04,  1.3231e-03,  2.7540

In [100]:
{opt: learner.clients[0].opt.state_dict()['state'].values() for opt, learner in learners.items()}#, learners['SGD'].clients[1].opt.state_dict()

{'Adadelta': dict_values([{'step': 600, 'square_avg': tensor([[[[9.1295e-06, 1.7709e-05, 7.0493e-05, 1.2797e-04, 1.2358e-04],
           [1.9307e-05, 6.4491e-05, 1.2109e-04, 1.6941e-04, 1.4161e-04],
           [5.4780e-05, 1.0876e-04, 1.6711e-04, 2.2641e-04, 1.0647e-04],
           [9.4926e-05, 1.3734e-04, 1.6936e-04, 1.3671e-04, 2.5744e-05],
           [1.0588e-04, 1.0795e-04, 9.0371e-05, 3.4470e-05, 4.7911e-06]]],
 
 
         [[[6.9240e-06, 1.5192e-05, 2.8659e-05, 1.9543e-05, 1.4967e-05],
           [1.1295e-05, 3.7958e-05, 4.9142e-05, 3.7909e-05, 1.7716e-05],
           [1.4169e-05, 4.8314e-05, 8.5488e-05, 5.8283e-05, 2.9175e-05],
           [1.3006e-05, 4.8754e-05, 8.1311e-05, 6.5554e-05, 3.2332e-05],
           [1.2284e-05, 3.5960e-05, 4.8460e-05, 4.8486e-05, 1.9683e-05]]],
 
 
         [[[3.6821e-06, 5.4744e-06, 4.3916e-06, 4.6502e-06, 2.7252e-05],
           [1.6873e-06, 1.2920e-06, 1.5877e-06, 1.1489e-05, 3.6392e-05],
           [3.9667e-06, 5.8406e-06, 3.4086e-06, 1.2084e-05,

In [98]:
{opt: learner.clients[0].opt.state_dict()['param_groups'] for opt, learner in learners.items()}#, learners['SGD'].clients[1].opt.state_dict()

{'Adadelta': [{'lr': 1e-06,
   'rho': 0.9,
   'eps': 1e-06,
   'weight_decay': 0,
   'params': [140520088536160,
    140520088536000,
    140520091289872,
    140520088538400,
    140522572013072,
    140520088134832,
    140520088555024,
    140520088555264]}],
 'Adagrad': [{'lr': 1e-06,
   'lr_decay': 0,
   'eps': 1e-10,
   'weight_decay': 0,
   'initial_accumulator_value': 0,
   'params': [140520087621296,
    140520087617936,
    140520087738640,
    140520088029296,
    140520088029856,
    140520088030976,
    140520088028656,
    140520088030816]}],
 'Adam': [{'lr': 1e-06,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0,
   'amsgrad': False,
   'params': [140520058870272,
    140520058870832,
    140520058867872,
    140520058870352,
    140520058868592,
    140520058871312,
    140521900644224,
    140520058828832]}],
 'AdamW': [{'lr': 1e-06,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'weight_decay': 0.01,
   'amsgrad': False,
   'params': [140520059851472,
 

In [81]:
[id(x) for x in learners['Adam'].clients[0].model.parameters()]#, [id(x) for x in learners['Adam'].clients[1].model.parameters()]

([140520058870272,
  140520058870832,
  140520058867872,
  140520058870352,
  140520058868592,
  140520058871312,
  140521900644224,
  140520058828832],
 [140520087972752,
  140520089115616,
  140520089115216,
  140520464606944,
  140520087751888,
  140520087748688,
  140520087751408,
  140520087749328])

In [80]:
[x.shape for x in learners['Adam'].clients[0].model.state_dict().values()], [x.shape for x in learners['Adam'].clients[1].model.state_dict().values()]

([torch.Size([32, 1, 5, 5]),
  torch.Size([32]),
  torch.Size([64, 32, 5, 5]),
  torch.Size([64]),
  torch.Size([512, 1024]),
  torch.Size([512]),
  torch.Size([10, 512]),
  torch.Size([10])],
 [torch.Size([32, 1, 5, 5]),
  torch.Size([32]),
  torch.Size([64, 32, 5, 5]),
  torch.Size([64]),
  torch.Size([512, 1024]),
  torch.Size([512]),
  torch.Size([10, 512]),
  torch.Size([10])])